In [ ]:
pip install fashion-clip

In [ ]:
pip install Pinecone

In [5]:
pip install more-itertools

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for more-itertools from https://files.pythonhosted.org/packages/2b/9f/7ba6f94fc1e9ac3d2b853fdff3035fb2fa5afbed898c4a72b8a020610594/more_itertools-10.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/65.3 kB ? eta -:--:--
   ---------------------------------------- 65.3/65.3 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from fashion_clip.fashion_clip import FashionCLIP

C:\Users\wendy\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np
import torch
import os
from dotenv import load_dotenv
from pinecone import Pinecone
import gc

In [3]:
# # Load the FashionCLIP model
fclip = FashionCLIP("fashion-clip")
# from utils.onnx_clip import fclip

In [4]:
# Pinecone 연결
load_dotenv()
api_key = os.getenv("API_KEY")
pc = Pinecone(api_key=api_key, environment="us-east-1")
tag_index = pc.Index("closhare-tags")  # 태그용 인덱스

In [7]:
fashion_tags = {
    "Clothing Types": [
        "T-shirt", "Hoodie", "Sweatshirt", "Jacket", "Coat", "Blazer", "Vest", "Sweater", "Cardigan",
        "Shirt", "Blouse", "Tank top", "Crop top",
        "Jeans", "Shorts", "Skirt", "Leggings", "Slacks", "Pants",
        "Dress", "Jumpsuit", "Overalls",
        "Underwear", "Lingerie", "Pajamas", "Sleepwear",
        "Swimwear", "Bikini", "Cover-up", "Rash guard",
        "Sneakers", "Running shoes", "Loafers", "Sandals", "Boots", "Heels",
        "Belt", "Scarf", "Tie", "Hat", "Cap", "Beanie", "Gloves", "Socks", "Tights", "Sunglasses",
        "Backpack", "Tote bag", "Clutch bag", "Crossbody bag", "Wallet"
    ],
    "Fashion Styles": [
        "Casual", "Streetwear", "Minimal", "Sporty", "Formal", "Business casual", "Chic", "Bohemian",
        "Vintage", "Retro", "Preppy", "Grunge", "Gothic", "Punk", "Athleisure", "Classic", "Eclectic",
        "Artsy", "Edgy", "Romantic", "Feminine", "Masculine", "Androgynous", "Avant-garde", "Luxury",
        "High fashion", "Urban", "Cozy", "Sophisticated", "Elegant", "Lovely", "Bold", "Subdued",
        "Timeless", "Transitional", "Resort", "Cruise", "Festival", "Beachwear", "Activewear",
        "Outdoor", "Travel", "Bridal"
    ],
    "Materials": [
        "Cotton", "Denim", "Leather", "Silk", "Wool", "Cashmere", "Linen", "Polyester", "Nylon",
        "Acrylic", "Rayon", "Viscose", "Spandex", "Bamboo", "Tencel", "Modal", "Faux fur",
        "Satin", "Velvet", "Corduroy", "Canvas", "Taffeta", "Chiffon", "Organza",
        "Jersey", "Sweatshirt fabric", "Fleece", "Terry cloth", "Neoprene", "Mesh", "Sheer",
        "Transparent", "Ribbed", "Quilted", "Puff print", "Brocade", "Jacquard", "Tartan"
    ],
    "Colors": [
        "Black", "White", "Gray", "Beige", "Red", "Blue", "Green", "Yellow", "Pink", "Purple", "Brown",
        "Orange", "Turquoise", "Navy", "Olive", "Burgundy", "Teal", "Coral", "Lavender", "Mustard",
        "Cream", "Maroon", "Charcoal", "Ivory", "Peach", "Mint", "Aqua", "Magenta", "Cyan", "Indigo",
        "Fuchsia", "Emerald", "Sapphire", "Ruby", "Amber", "Copper", "Bronze", "Silver", "Gold",
        "Platinum", "Rose gold", "Champagne", "Slate", "Dusty rose", "Seafoam", "Lilac", "Berry",
        "Rust", "Terracotta"
    ],
    "Seasons": ["Spring", "Summer", "Fall", "Winter"]
}


In [ ]:
# with torch.no_grad():
#     text_vec = fclip.encode_text(["A photo of jacket"], batch_size=1)[0]
# print("Text vector shape:", text_vec.shape)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

100%|██████████| 1/1 [00:00<00:00, 17.66it/s]

Text vector shape: (512,)


In [ ]:
# from PIL import Image

# image_path = "1.jpg"
# image = Image.open(image_path).convert("RGB")
# with torch.no_grad():
#     image_vec = fclip.encode_images([image], batch_size=1)[0]
# print("Image vector shape:", image_vec.shape)


100%|██████████| 1/1 [00:00<00:00,  7.67it/s]

Image vector shape: (512,)


In [9]:
from more_itertools import chunked  # pip install more-itertools

# 기존 레코드를 모두 지우고 새로 시작 (원하면 주석 처리)
tag_index.delete(delete_all=True)

all_texts = []
id_metadata_pairs = []

for category, category_tags in fashion_tags.items():
    for tag in category_tags:
        all_texts.append(f"A photo of {tag}")
        id_metadata_pairs.append((tag, category))

batch_size = 32  # 시스템에 맞게 조정

for texts_chunk, meta_chunk in zip(chunked(all_texts, batch_size), chunked(id_metadata_pairs, batch_size)):
    with torch.no_grad():
        vectors = fclip.encode_text(texts_chunk, batch_size=batch_size)
        vectors /= np.linalg.norm(vectors, axis=1, keepdims=True)

    upload_batch = []
    for (tag, category), vector in zip(meta_chunk, vectors):
        upload_batch.append({
            "id": tag,
            "values": vector.tolist(),
            "metadata": {
                "tag": tag,
                "category": category
            }
        })

    tag_index.upsert(upload_batch)
    print(f"[UPLOADED] {len(upload_batch)} tags")

100%|██████████| 1/1 [00:01<00:00,  1.57s/it]


[UPLOADED] 32 tags


100%|██████████| 1/1 [00:01<00:00,  1.53s/it]


[UPLOADED] 32 tags


100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


...

[UPLOADED] 32 tags


100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


[UPLOADED] 32 tags


Map: 100%|██████████| 25/25 [00:00<00:00, 3456.08 examples/s]
1it [00:01,  1.24s/it]


[UPLOADED] 25 tags


In [ ]:
# # onnx_clip 사용

# """
# closhare-tags 인덱스 재빌드
# -------------------------------------------------
# 환경 변수
#   API_KEY=<PINECONE_API_KEY>

# 필요 패키지
#   pip install pinecone-client more-itertools python-dotenv
# """

# import os
# from more_itertools import chunked
# from pinecone import Pinecone

# import numpy as np
# from utils.onnx_clip import fclip          # ← 방금 수정한 클래스
# from utils.labels import fashion_tags_kr   # 한·영 매핑
# from utils.onnx_clip import fclip

# # 0. Pinecone 세션 -------------------------------------------------------
# pc = Pinecone(api_key=os.getenv("API_KEY"), environment="us-east-1")
# tag_index = pc.Index("closhare-tags")

# # 기존 레코드를 모두 지우고 새로 시작 (원하면 주석 처리)
# tag_index.delete(delete_all=True)

# # 1. 카테고리별 태그 딕셔너리가 이미 있다면 그대로 사용
# #    없다면 fashion_tags_kr.keys()에서 직접 만들 수 있습니다.
# #    여기서는 예시로 미리 준비된 'fashion_tags' dict를 가정합니다.    # {category: [tag_en, ...]}

# # 2. 업서트 --------------------------------------------------------------
# BATCH = 32

# all_texts = []
# id_meta_pairs = []

# for category, tag_list in fashion_tags.items():
#     for tag_en in tag_list:
#         all_texts.append(f"A photo of {tag_en}")
#         id_meta_pairs.append((tag_en, category))

# for texts_chunk, meta_chunk in zip(
#         chunked(all_texts, BATCH), chunked(id_meta_pairs, BATCH)):

#     # 이미 L2-정규화된 FP32 벡터 반환
#     vectors = fclip.encode_text(texts_chunk)

#     upload_batch = []
#     for (tag_en, category), vec in zip(meta_chunk, vectors):
#         upload_batch.append({
#             "id": tag_en,
#             "values": vec.tolist(),
#             "metadata": {
#                 "tag": tag_en,
#                 "category": category
#             }
#         })

#     tag_index.upsert(upload_batch)
#     print(f"[UPLOADED] {len(upload_batch)} tags")

# print("✅  tag-index rebuild complete!")

[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 32 tags
[UPLOADED] 6 tags
✅  tag-index rebuild complete!


In [10]:
import os
from PIL import Image
from io import BytesIO

from utils.onnx_clip import fclip

# Load and encode image
image_path = "1_006.jpg"
image = Image.open(image_path).convert("RGB")
with torch.no_grad():
    vector = fclip.encode_images([image])[0]
    vector /= np.linalg.norm(vector)

# Define categories (same keys as fashion_tags)
categories = [
    "Clothing Types", "Fashion Styles",
    "Materials", "Colors", "Seasons"
]

# Search top tag per category
results = []

for category in categories:
    query_result = tag_index.query(
        vector=vector.tolist(),
        top_k=1,
        include_metadata=True,
        filter={"category": {"$eq": category}}  # Only search in this category
    )
    matches = query_result.get("matches", [])
    print(matches)
    if matches:
        match = matches[0]
        results.append({
            "category": category,
            "tag": match["metadata"]["tag"],
            "score": match["score"]
        })
    else:
        results.append({
            "category": category,
            "tag": None,
            "score": None
        })

# Print results
for r in results:
    print(f"[{r['category']}] → {r['tag']} (score: {r['score']:.4f})" if r['tag'] else f"[{r['category']}] → No match")

[{'id': 'Crop top',
 'metadata': {'category': 'Clothing Types', 'tag': 'Crop top'},
 'score': 0.330635071,
 'values': []}]
[{'id': 'Vintage',
 'metadata': {'category': 'Fashion Styles', 'tag': 'Vintage'},
 'score': 0.266296357,
 'values': []}]
[{'id': 'Sweatshirt fabric',
 'metadata': {'category': 'Materials', 'tag': 'Sweatshirt fabric'},
 'score': 0.271499664,
 'values': []}]
[{'id': 'Yellow',
 'metadata': {'category': 'Colors', 'tag': 'Yellow'},
 'score': 0.267311096,
 'values': []}]
[{'id': 'Summer',
 'metadata': {'category': 'Seasons', 'tag': 'Summer'},
 'score': 0.227600127,
 'values': []}]
[Clothing Types] → Crop top (score: 0.3306)
[Fashion Styles] → Vintage (score: 0.2663)
[Materials] → Sweatshirt fabric (score: 0.2715)
[Colors] → Yellow (score: 0.2673)
[Seasons] → Summer (score: 0.2276)


In [7]:
vector

array([-3.24518583e-03, -3.99320968e-04, -2.13721097e-02, -1.68108512e-02,
        4.75191958e-02, -1.98029447e-02, -4.14900109e-02,  1.29469978e-02,
        3.01852897e-02,  7.93130174e-02, -2.27219239e-03, -1.20077506e-02,
       -3.66306677e-02,  1.99323036e-02, -1.04104662e-02, -1.07535450e-02,
        4.15631244e-03, -1.87005959e-02,  1.95048619e-02,  4.85371845e-03,
       -1.46680158e-02, -1.05510727e-02,  1.99716724e-02,  4.85371845e-03,
       -1.82675272e-02, -1.19458828e-02,  1.84981227e-02,  4.02189307e-02,
       -6.13604486e-03, -5.36552444e-03, -2.84980182e-02,  1.17659075e-02,
       -1.84362568e-02,  2.63776816e-02,  8.29012785e-03,  2.05453448e-02,
        2.17883028e-02, -2.34530754e-02, -1.08435331e-02, -1.56522579e-02,
       -4.28567035e-03, -3.67544033e-02,  2.44991854e-02,  8.08765553e-03,
       -5.29240910e-03, -4.02358063e-02, -3.45215797e-02,  1.71932988e-02,
       -3.25924642e-02, -2.33237203e-02, -1.67433582e-02, -2.18557920e-02,
        3.99545915e-02,  